# AI-Powered Intelligence for Agile Teams
### 1. Data Understanding

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sentistrength import PySentiStr
import re
import yaml
import os

In [ ]:
# Initialize analyzers
vader_analyzer = SentimentIntensityAnalyzer()
sentistrength = PySentiStr()

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
# Load configuration from config.yaml
# Determine project root based on current working directory
# If we're in src/, go up one level; otherwise assume we're at project root
current_dir = os.getcwd()
if os.path.basename(current_dir) == 'src':
    project_root = os.path.dirname(current_dir)
else:
    project_root = current_dir

config_path = os.path.join(project_root, 'config', 'config.yaml')

# Verify config file exists
if not os.path.exists(config_path):
    raise FileNotFoundError(
        f"Config file not found at: {config_path}\n"
        f"Current working directory: {current_dir}\n"
        f"Project root: {project_root}\n"
        f"Please ensure you're running the notebook from the project root or src/ directory."
    )

# Read config file
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Extract data file path from config
data_file_path = config['data']['file_path']
data_file_type = config['data'].get('file_type', 'auto')

# Resolve data file path relative to project root
if not os.path.isabs(data_file_path):
    # Handle relative paths (e.g., "../data/file.xlsx" or "data/file.xlsx")
    # Remove "../" prefix and join to project root
    path_clean = data_file_path.replace('../', '') if data_file_path.startswith('../') else data_file_path
    data_file_path = os.path.join(project_root, path_clean)

# Verify data file exists
if not os.path.exists(data_file_path):
    raise FileNotFoundError(
        f"Data file not found at: {data_file_path}\n"
        f"Please check the 'file_path' setting in config.yaml"
    )

# Display loaded configuration
print("Configuration loaded successfully!")
print(f"Project root: {project_root}")
print(f"Config file: {config_path}")
print(f"Data file path: {data_file_path}")
print(f"Data file type: {data_file_type}")

# Store LLM config for future use
llm_config = config.get('llm', {})
print(f"LLM Provider: {llm_config.get('provider', 'none')}")

In [ ]:
# Read the file using path from config
# Determine file type if auto-detection is enabled
if data_file_type == 'auto':
    # Auto-detect from file extension
    file_ext = os.path.splitext(data_file_path)[1].lower()
    if file_ext in ['.xlsx', '.xls']:
        data_file_type = 'excel'
    elif file_ext == '.csv':
        data_file_type = 'csv'
    else:
        raise ValueError(f"Unsupported file type: {file_ext}. Please specify file_type in config.")

# Read file based on detected/configured type
if data_file_type == 'excel':
    readfile = pd.read_excel(data_file_path)
elif data_file_type == 'csv':
    readfile = pd.read_csv(data_file_path)
else:
    raise ValueError(f"Invalid file_type in config: {data_file_type}. Must be 'excel', 'csv', or 'auto'.")

print(f"Successfully loaded data from: {data_file_path}")
print(f"Data shape: {readfile.shape}")

In [ ]:
readfile

In [ ]:
readfile.columns.tolist()

In [ ]:
len(readfile.columns)

In [ ]:
len(readfile)

In [ ]:
# new = readfile[['Issue.id', 'Issue.Type', 'Priority', 'Project.type', 'Status', 'Created', 'Resolution', 'Resolved', 'Component.s', 'Custom.field..Symptom.Severity.', 'Custom.field..Company.', 'Custom.field..Date.of.First.Response.', 'Votes', 'Environment', 'Custom.field..Original.story.points.']].copy()
new = readfile[['Summary', 'Issue key', 'Issue Type', 'Status', 'Created', 'Resolved', 'Comment', 'Comment.1', 'Comment.2', 'Comment.3', 'Comment.4', 'Comment.5', 'Comment.6', 'Comment.7', 'Comment.8', 'Comment.9', 'Comment.10', 'Comment.11', 'Comment.12', 'Comment.13', 'Comment.14', 'Comment.15', 'Comment.16', 'Comment.17', 'Comment.18', 'Comment.19', 'Comment.20', 'Comment.21', 'Comment.22', 'Comment.23', 'Comment.24', 'Comment.25', 'Comment.26', 'Comment.27', 'Comment.28', 'Comment.29', 'Comment.30', 'Comment.31', 'Comment.32', 'Comment.33', 'Comment.34', 'Comment.35', 'Comment.36', 'Comment.37', 'Comment.38', 'Comment.39', 'Comment.40', 'Comment.41', 'Comment.42', 'Comment.43', 'Comment.44', 'Comment.45', 'Comment.46', 'Comment.47', 'Comment.48', 'Comment.49', 'Comment.50', 'Comment.51', 'Comment.52', 'Comment.53', 'Comment.54', 'Comment.55', 'Comment.56', 'Comment.57', 'Comment.58', 'Comment.59', 'Comment.60', 'Comment.61', 'Comment.62', 'Comment.63', 'Comment.64', 'Comment.65', 'Comment.66', 'Comment.67', 'Comment.68', 'Comment.69', 'Comment.70', 'Comment.71', 'Comment.72', 'Comment.73', 'Comment.74', 'Comment.75', 'Comment.76', 'Comment.77', 'Comment.78', 'Comment.79', 'Comment.80', 'Comment.81', 'Comment.82', 'Comment.83', 'Comment.84']].copy()
new

In [ ]:
plt.figure(figsize=(10, 6))
new['Issue Type'].value_counts().plot(kind='bar')
plt.title('Issue Type Distribution')
plt.xlabel('Issue Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 2. Sentiment Analysis on Comment Columns


### 4.1 Improved Sentiment Analysis Setup


In [ ]:
# Improved text preprocessing function
def preprocess_text(text):
    """
    Clean and preprocess text for better sentiment analysis.
    """
    if pd.isna(text) or str(text).strip() == '':
        return ''
    
    text = str(text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove special characters but keep punctuation that affects sentiment (!, ?, .)
    text = re.sub(r'[^\w\s!?.]', ' ', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    # Remove timestamps and dates in common formats
    text = re.sub(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', '', text)
    text = re.sub(r'\d{1,2}:\d{2}(?:\s*[AP]M)?', '', text, flags=re.IGNORECASE)
    
    # Remove user IDs and hashes (common in JIRA comments)
    text = re.sub(r'[a-f0-9]{32,}', '', text)  # Remove long hex strings
    
    return text.strip()


In [ ]:
# Improved sentiment analysis functions

def analyze_sentiment_textblob(text):
    """Original TextBlob analysis"""
    if pd.isna(text) or str(text).strip() == '':
        return (np.nan, np.nan, 'neutral')
    
    try:
        cleaned_text = preprocess_text(text)
        if not cleaned_text:
            return (np.nan, np.nan, 'neutral')
        
        blob = TextBlob(cleaned_text)
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity
        
        if polarity > 0.1:
            sentiment_label = 'positive'
        elif polarity < -0.1:
            sentiment_label = 'negative'
        else:
            sentiment_label = 'neutral'
        
        return (polarity, subjectivity, sentiment_label)
    except:
        return (np.nan, np.nan, 'neutral')

def analyze_sentiment_vader(text):
    """VADER sentiment analysis (better for short texts)"""
    if pd.isna(text) or str(text).strip() == '':
        return (np.nan, np.nan, 'neutral')
    
    try:
        cleaned_text = preprocess_text(text)
        if not cleaned_text:
            return (np.nan, np.nan, 'neutral')
        
        scores = vader_analyzer.polarity_scores(cleaned_text)
        polarity = scores['compound']  # VADER compound score ranges from -1 to 1
        
        # VADER doesn't provide subjectivity, so we use a simple heuristic
        subjectivity = (scores['pos'] + scores['neg']) / 2
        
        # Classify sentiment
        if polarity > 0.05:
            sentiment_label = 'positive'
        elif polarity < -0.05:
            sentiment_label = 'negative'
        else:
            sentiment_label = 'neutral'
        
        return (polarity, subjectivity, sentiment_label)
    except:
        return (np.nan, np.nan, 'neutral')

def analyze_sentiment_sentistrength(text):
    """SentiStrength-SE sentiment analysis (designed for software engineering texts)"""
    if pd.isna(text) or str(text).strip() == '':
        return (np.nan, np.nan, 'neutral')
    
    try:
        cleaned_text = preprocess_text(text)
        if not cleaned_text:
            return (np.nan, np.nan, 'neutral')
        
        # SentiStrength returns positive and negative scores separately
        # Format: [positive_score, negative_score] where scores range from 1-5
        result = sentistrength.getSentiment(cleaned_text)
        
        if result and len(result) >= 2:
            pos_score = result[0]  # Positive score (1-5)
            neg_score = result[1]  # Negative score (1-5)
            
            # Convert to polarity scale (-1 to 1)
            # SentiStrength: 1-5 scale, we normalize to -1 to 1
            # Positive sentiment: pos_score > neg_score
            # Negative sentiment: neg_score > pos_score
            if pos_score > neg_score:
                # Positive sentiment: map 1-5 to 0.2-1.0
                polarity = (pos_score - 1) / 4.0  # Maps 1->0, 5->1, but we want 0.2-1.0
                polarity = 0.2 + (polarity * 0.8)  # Scale to 0.2-1.0
            elif neg_score > pos_score:
                # Negative sentiment: map 1-5 to -1.0 to -0.2
                polarity = -((neg_score - 1) / 4.0)  # Maps 1->0, 5->-1, but we want -1.0 to -0.2
                polarity = -0.2 + (polarity * 0.8)  # Scale to -1.0 to -0.2
            else:
                # Neutral
                polarity = 0.0
            
            # Subjectivity: higher when both scores are away from 1 (neutral)
            max_score = max(pos_score, neg_score)
            subjectivity = (max_score - 1) / 4.0  # Normalize to 0-1
            
            # Classify sentiment
            if polarity > 0.1:
                sentiment_label = 'positive'
            elif polarity < -0.1:
                sentiment_label = 'negative'
            else:
                sentiment_label = 'neutral'
            
            return (polarity, subjectivity, sentiment_label)
        else:
            return (np.nan, np.nan, 'neutral')
    except Exception as e:
        # If SentiStrength fails, return neutral
        return (np.nan, np.nan, 'neutral')

def analyze_sentiment_ensemble(text):
    """
    Ensemble method: Combine TextBlob, VADER, and SentiStrength-SE for better accuracy.
    Uses weighted average of all available methods.
    SentiStrength-SE is specifically designed for software engineering texts (like JIRA comments).
    """
    if pd.isna(text) or str(text).strip() == '':
        return (np.nan, np.nan, 'neutral')
    
    tb_result = analyze_sentiment_textblob(text)
    vd_result = analyze_sentiment_vader(text)
    ss_result = analyze_sentiment_sentistrength(text)
    
    # Collect valid results
    valid_results = []
    weights = []
    
    if not pd.isna(tb_result[0]):
        valid_results.append(tb_result)
        weights.append(0.25)  # TextBlob: 25% weight
    
    if not pd.isna(vd_result[0]):
        valid_results.append(vd_result)
        weights.append(0.35)  # VADER: 35% weight (good for short texts)
    
    if not pd.isna(ss_result[0]):
        valid_results.append(ss_result)
        weights.append(0.40)  # SentiStrength-SE: 40% weight (designed for SE texts)
    
    # If no valid results, return neutral
    if len(valid_results) == 0:
        return (np.nan, np.nan, 'neutral')
    
    # Normalize weights to sum to 1.0
    total_weight = sum(weights)
    weights = [w / total_weight for w in weights]
    
    # Weighted combination
    combined_polarity = sum(r[0] * w for r, w in zip(valid_results, weights))
    combined_subjectivity = sum(r[1] * w for r, w in zip(valid_results, weights))
    
    # Classify based on combined polarity
    if combined_polarity > 0.1:
        sentiment_label = 'positive'
    elif combined_polarity < -0.1:
        sentiment_label = 'negative'
    else:
        sentiment_label = 'neutral'
    
    return (combined_polarity, combined_subjectivity, sentiment_label)

# Choose analysis method: 'textblob', 'vader', 'sentistrength', or 'ensemble'
SENTIMENT_METHOD = 'ensemble'  # Change this to use different methods
print(f"Using sentiment analysis method: {SENTIMENT_METHOD}")
if SENTIMENT_METHOD == 'ensemble':
    print("Ensemble combines: TextBlob (25%), VADER (35%), SentiStrength-SE (40%)")

# Map method name to function
sentiment_methods = {
    'textblob': analyze_sentiment_textblob,
    'vader': analyze_sentiment_vader,
    'sentistrength': analyze_sentiment_sentistrength,
    'ensemble': analyze_sentiment_ensemble
}

analyze_sentiment = sentiment_methods.get(SENTIMENT_METHOD, analyze_sentiment_ensemble)


In [ ]:
# Compare different sentiment analysis methods on sample comments
print("="*80)
print("COMPARING SENTIMENT ANALYSIS METHODS")
print("="*80)

# Identify comment columns first (needed for comparison)
comment_columns = [col for col in new.columns if col.startswith('Comment')]

# Get some sample comments for comparison
sample_comments = []
for col in comment_columns[:5]:  # Use first 5 comment columns
    non_empty = new[col].dropna()
    non_empty = non_empty[non_empty.astype(str).str.strip() != '']
    if len(non_empty) > 0:
        sample_comments.extend(non_empty.head(3).tolist())
    if len(sample_comments) >= 10:
        break

print(f"\nComparing {len(sample_comments)} sample comments:\n")

comparison_results = []
for i, comment in enumerate(sample_comments[:10], 1):
    tb = analyze_sentiment_textblob(comment)
    vd = analyze_sentiment_vader(comment)
    ss = analyze_sentiment_sentistrength(comment)
    en = analyze_sentiment_ensemble(comment)
    
    comparison_results.append({
        'comment': comment[:100] + '...' if len(comment) > 100 else comment,
        'textblob_polarity': tb[0],
        'textblob_label': tb[2],
        'vader_polarity': vd[0],
        'vader_label': vd[2],
        'sentistrength_polarity': ss[0],
        'sentistrength_label': ss[2],
        'ensemble_polarity': en[0],
        'ensemble_label': en[2]
    })

comparison_df = pd.DataFrame(comparison_results)
print(comparison_df.to_string(index=False))

print("\n" + "="*80)
print("Method Comparison Summary:")
print(f"TextBlob and VADER agree: {sum(comparison_df['textblob_label'] == comparison_df['vader_label'])}/{len(comparison_df)}")
print(f"TextBlob and SentiStrength-SE agree: {sum(comparison_df['textblob_label'] == comparison_df['sentistrength_label'])}/{len(comparison_df)}")
print(f"VADER and SentiStrength-SE agree: {sum(comparison_df['vader_label'] == comparison_df['sentistrength_label'])}/{len(comparison_df)}")
print(f"TextBlob and Ensemble agree: {sum(comparison_df['textblob_label'] == comparison_df['ensemble_label'])}/{len(comparison_df)}")
print(f"VADER and Ensemble agree: {sum(comparison_df['vader_label'] == comparison_df['ensemble_label'])}/{len(comparison_df)}")
print(f"SentiStrength-SE and Ensemble agree: {sum(comparison_df['sentistrength_label'] == comparison_df['ensemble_label'])}/{len(comparison_df)}")
print("="*80)


### 4.2 Sentiment Analysis Execution (Using Improved Method)

**Improvements Made:**
1. **Text Preprocessing**: Removes URLs, emails, timestamps, and noise that can affect sentiment analysis
2. **VADER Sentiment Analyzer**: Better for short texts and social media-style comments (common in JIRA)
3. **Ensemble Method**: Combines TextBlob and VADER (60% VADER, 40% TextBlob) for improved accuracy
4. **Method Selection**: Easy to switch between methods by changing `SENTIMENT_METHOD` variable

**To change the method**, edit cell 12 and set:
- `SENTIMENT_METHOD = 'textblob'` - Original TextBlob only
- `SENTIMENT_METHOD = 'vader'` - VADER only (recommended for short texts)
- `SENTIMENT_METHOD = 'ensemble'` - Combined method (recommended for best accuracy)


In [ ]:
# Identify all comment columns (columns that start with "Comment")
comment_columns = [col for col in new.columns if col.startswith('Comment')]
print(f"Found {len(comment_columns)} comment columns:")
print(comment_columns)


In [ ]:
# Function to analyze sentiment of a text
def analyze_sentiment(text):
    """
    Analyze sentiment of a text string.
    Returns: (polarity, subjectivity, sentiment_label)
    - polarity: ranges from -1 (negative) to 1 (positive)
    - subjectivity: ranges from 0 (objective) to 1 (subjective)
    - sentiment_label: 'positive', 'negative', or 'neutral'
    """
    if pd.isna(text) or str(text).strip() == '':
        return (np.nan, np.nan, 'neutral')
    
    try:
        blob = TextBlob(str(text))
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity
        
        # Classify sentiment
        if polarity > 0.1:
            sentiment_label = 'positive'
        elif polarity < -0.1:
            sentiment_label = 'negative'
        else:
            sentiment_label = 'neutral'
        
        return (polarity, subjectivity, sentiment_label)
    except:
        return (np.nan, np.nan, 'neutral')


In [ ]:
# Perform sentiment analysis on all comment columns
print("Performing sentiment analysis on comment columns...")
sentiment_results = {}
new_columns_dict = {}

for col in comment_columns:
    print(f"Processing {col}...")
    sentiments = new[col].apply(analyze_sentiment)
    
    # Extract polarity, subjectivity, and labels and Store results in a dictionary instead of assigning to DataFrame directly
    new_columns_dict[f'{col}_polarity'] = sentiments.apply(lambda x: x[0])
    new_columns_dict[f'{col}_subjectivity'] = sentiments.apply(lambda x: x[1])
    new_columns_dict[f'{col}_sentiment'] = sentiments.apply(lambda x: x[2])
    
    # Store summary statistics
    sentiment_results[col] = {
        'total_comments': new[col].notna().sum(),
        'non_empty_comments': (new[col].notna() & (new[col].astype(str).str.strip() != '')).sum(),
        'avg_polarity': new_columns_dict[f'{col}_polarity'].mean(),
        'avg_subjectivity': new_columns_dict[f'{col}_subjectivity'].mean(),
        'positive_count': (new_columns_dict[f'{col}_sentiment'] == 'positive').sum(),
        'negative_count': (new_columns_dict[f'{col}_sentiment'] == 'negative').sum(),
        'neutral_count': (new_columns_dict[f'{col}_sentiment'] == 'neutral').sum()
    }

# Concatenate all new columns at once to avoid DataFrame fragmentation
new = pd.concat([new, pd.DataFrame(new_columns_dict)], axis=1)

print("Sentiment analysis completed!")

In [ ]:
# Create a summary dataframe for all comment columns
sentiment_summary = pd.DataFrame(sentiment_results).T
sentiment_summary = sentiment_summary.sort_values('non_empty_comments', ascending=False)
print("Sentiment Analysis Summary for All Comment Columns:")
print(sentiment_summary)


In [ ]:
# Aggregate all comments into a single analysis
# Combine all comment columns into one series for overall analysis
all_comments = []
for col in comment_columns:
    non_empty = new[col].dropna()
    non_empty = non_empty[non_empty.astype(str).str.strip() != '']
    all_comments.extend(non_empty.tolist())

print(f"Total non-empty comments across all columns: {len(all_comments)}")

# Analyze overall sentiment
overall_sentiments = [analyze_sentiment(comment) for comment in all_comments]
overall_polarities = [s[0] for s in overall_sentiments if not np.isnan(s[0])]
overall_subjectivities = [s[1] for s in overall_sentiments if not np.isnan(s[1])]
overall_labels = [s[2] for s in overall_sentiments]

print(f"\nOverall Sentiment Statistics:")
print(f"Average Polarity: {np.mean(overall_polarities):.4f}")
print(f"Average Subjectivity: {np.mean(overall_subjectivities):.4f}")
print(f"\nSentiment Distribution:")
print(f"Positive: {overall_labels.count('positive')} ({overall_labels.count('positive')/len(overall_labels)*100:.2f}%)")
print(f"Negative: {overall_labels.count('negative')} ({overall_labels.count('negative')/len(overall_labels)*100:.2f}%)")
print(f"Neutral: {overall_labels.count('neutral')} ({overall_labels.count('neutral')/len(overall_labels)*100:.2f}%)")


In [ ]:
# Prepare comments dataframe for displaying top comments
# Create a dataframe with comments and their sentiments
comments_with_sentiment = []
for col in comment_columns:
    for idx in new.index:
        comment = new.loc[idx, col]
        if pd.notna(comment) and str(comment).strip() != '':
            sentiment_col = f'{col}_sentiment'
            polarity_col = f'{col}_polarity'
            sentiment = new.loc[idx, sentiment_col]
            polarity = new.loc[idx, polarity_col]
            if pd.notna(sentiment) and pd.notna(polarity):
                comments_with_sentiment.append({
                    'comment': str(comment),
                    'sentiment': sentiment,
                    'polarity': polarity,
                    'column': col
                })

comments_df = pd.DataFrame(comments_with_sentiment)
num_samples = 15
print(f"Prepared {len(comments_df)} comments with sentiment analysis for display.")


In [ ]:
# Display Top Positive Comments
print("="*80)
print(f"TOP {num_samples} POSITIVE COMMENTS (Highest Polarity)")
print("="*80)
positive_comments = comments_df[comments_df['sentiment'] == 'positive'].sort_values('polarity', ascending=False)
if len(positive_comments) > 0:
    for i, (idx, row) in enumerate(positive_comments.head(num_samples).iterrows(), 1):
        print(f"\n[{i}] Polarity: {row['polarity']:.4f} | Column: {row['column']}")
        print(f"Comment: {row['comment'][:600]}")  # Limit to 600 chars
        if len(row['comment']) > 600:
            print("... (truncated)")
else:
    print("No positive comments found.")
print("\n" + "="*80)


In [ ]:
# Display Top Negative Comments
print("="*80)
print(f"TOP {num_samples} NEGATIVE COMMENTS (Lowest Polarity)")
print("="*80)
negative_comments = comments_df[comments_df['sentiment'] == 'negative'].sort_values('polarity', ascending=True)
if len(negative_comments) > 0:
    for i, (idx, row) in enumerate(negative_comments.head(num_samples).iterrows(), 1):
        print(f"\n[{i}] Polarity: {row['polarity']:.4f} | Column: {row['column']}")
        print(f"Comment: {row['comment'][:600]}")  # Limit to 600 chars
        if len(row['comment']) > 600:
            print("... (truncated)")
else:
    print("No negative comments found.")
print("\n" + "="*80)


In [ ]:
# Display Top Neutral Comments
print("="*80)
print(f"TOP {num_samples} NEUTRAL COMMENTS (Closest to Zero)")
print("="*80)
neutral_comments = comments_df[comments_df['sentiment'] == 'neutral'].copy()
neutral_comments['abs_polarity'] = neutral_comments['polarity'].abs()
neutral_comments = neutral_comments.sort_values('abs_polarity', ascending=True)
if len(neutral_comments) > 0:
    for i, (idx, row) in enumerate(neutral_comments.head(num_samples).iterrows(), 1):
        print(f"\n[{i}] Polarity: {row['polarity']:.4f} | Column: {row['column']}")
        print(f"Comment: {row['comment'][:600]}")  # Limit to 600 chars
        if len(row['comment']) > 600:
            print("... (truncated)")
else:
    print("No neutral comments found.")
print("\n" + "="*80)


In [ ]:
# More detailed sample comments with better formatting
print("\n" + "="*80)
print("DETAILED TOP COMMENTS BY SENTIMENT CATEGORY")
print("="*80)

# Number of top samples to display per category
num_samples_detailed = 15

# Function to get top comments efficiently across all columns
def get_top_comments(sentiment_type, num_samples=15):
    """Get top comments of a specific sentiment type across all columns"""
    samples = []
    for col in comment_columns:
        sentiment_col = f'{col}_sentiment'
        polarity_col = f'{col}_polarity'
        
        # Get all matching comments from this column
        mask = (new[sentiment_col] == sentiment_type) & new[col].notna()
        matching_data = new[mask][[col, polarity_col]].copy()
        
        for idx, row in matching_data.iterrows():
            comment = str(row[col]).strip()
            if comment and comment != '':
                polarity = row[polarity_col]
                if pd.notna(polarity):
                    samples.append({
                        'comment': comment,
                        'polarity': polarity,
                        'column': col,
                        'index': idx
                    })
    
    # Convert to DataFrame and sort
    if len(samples) == 0:
        return pd.DataFrame()
    
    samples_df = pd.DataFrame(samples)
    
    # Sort based on sentiment type
    if sentiment_type == 'positive':
        # Sort by highest polarity (most positive first)
        samples_df = samples_df.sort_values('polarity', ascending=False)
    elif sentiment_type == 'negative':
        # Sort by lowest polarity (most negative first)
        samples_df = samples_df.sort_values('polarity', ascending=True)
    else:  # neutral
        # Sort by absolute polarity (closest to zero first)
        samples_df['abs_polarity'] = samples_df['polarity'].abs()
        samples_df = samples_df.sort_values('abs_polarity', ascending=True)
    
    return samples_df.head(num_samples)

# Get and display positive comments
print("\n" + "-"*80)
print(f"TOP {num_samples_detailed} POSITIVE COMMENTS (Highest Polarity)")
print("-"*80)
pos_samples = get_top_comments('positive', num_samples_detailed)
if len(pos_samples) > 0:
    for i, (idx, row) in enumerate(pos_samples.iterrows(), 1):
        print(f"\n[Sample {i}]")
        print(f"  Polarity Score: {row['polarity']:.4f}")
        print(f"  Source Column: {row['column']}")
        print(f"  Comment:")
        # Print comment with proper word wrapping
        comment_text = row['comment']
        if len(comment_text) > 800:
            print(f"  {comment_text[:800]}...")
        else:
            print(f"  {comment_text}")
else:
    print("No positive comments found.")

# Get and display negative comments
print("\n" + "-"*80)
print(f"TOP {num_samples_detailed} NEGATIVE COMMENTS (Lowest Polarity)")
print("-"*80)
neg_samples = get_top_comments('negative', num_samples_detailed)
if len(neg_samples) > 0:
    for i, (idx, row) in enumerate(neg_samples.iterrows(), 1):
        print(f"\n[Sample {i}]")
        print(f"  Polarity Score: {row['polarity']:.4f}")
        print(f"  Source Column: {row['column']}")
        print(f"  Comment:")
        comment_text = row['comment']
        if len(comment_text) > 800:
            print(f"  {comment_text[:800]}...")
        else:
            print(f"  {comment_text}")
else:
    print("No negative comments found.")

# Get and display neutral comments
print("\n" + "-"*80)
print(f"TOP {num_samples_detailed} NEUTRAL COMMENTS (Closest to Zero Polarity)")
print("-"*80)
neu_samples = get_top_comments('neutral', num_samples_detailed)
if len(neu_samples) > 0:
    for i, (idx, row) in enumerate(neu_samples.iterrows(), 1):
        print(f"\n[Sample {i}]")
        print(f"  Polarity Score: {row['polarity']:.4f}")
        print(f"  Source Column: {row['column']}")
        print(f"  Comment:")
        comment_text = row['comment']
        if len(comment_text) > 800:
            print(f"  {comment_text[:800]}...")
        else:
            print(f"  {comment_text}")
else:
    print("No neutral comments found.")

print("\n" + "="*80)


In [ ]:
# Visualization 1: Overall Sentiment Distribution
plt.figure(figsize=(10, 6))
sentiment_counts = pd.Series(overall_labels).value_counts()
colors = {'positive': 'green', 'negative': 'red', 'neutral': 'gray'}
sentiment_counts.plot(kind='bar', color=[colors.get(x, 'blue') for x in sentiment_counts.index])
plt.title('Overall Sentiment Distribution Across All Comments')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# Visualization 2: Polarity Distribution Histogram
plt.figure(figsize=(12, 6))
plt.hist(overall_polarities, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Neutral')
plt.title('Distribution of Sentiment Polarity Scores')
plt.xlabel('Polarity Score (Negative to Positive)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Visualization 3: Average Polarity by Comment Column (Top 20 columns with most comments)
top_columns = sentiment_summary.head(20)
plt.figure(figsize=(14, 8))
plt.barh(range(len(top_columns)), top_columns['avg_polarity'], 
         color=['green' if x > 0 else 'red' if x < 0 else 'gray' for x in top_columns['avg_polarity']])
plt.yticks(range(len(top_columns)), top_columns.index)
plt.xlabel('Average Polarity Score')
plt.title('Average Sentiment Polarity by Comment Column (Top 20 by Comment Count)')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()


In [ ]:
# Visualization 4: Sentiment Distribution by Comment Column (Top 10)
top_10_columns = sentiment_summary.head(10).index
fig, axes = plt.subplots(2, 5, figsize=(20, 10))
axes = axes.flatten()

# Define color mapping for sentiment labels
sentiment_colors = {'positive': 'green', 'negative': 'red', 'neutral': 'gray'}

for idx, col in enumerate(top_10_columns):
    sentiment_col = f'{col}_sentiment'
    sentiment_dist = new[sentiment_col].value_counts()
    colors = [sentiment_colors.get(x, 'blue') for x in sentiment_dist.index]
    sentiment_dist.plot(kind='bar', ax=axes[idx], color=colors, rot=45)
    axes[idx].set_title(f'{col}\n({sentiment_summary.loc[col, "non_empty_comments"]} comments)')
    axes[idx].set_xlabel('')
    axes[idx].set_ylabel('Count')

plt.suptitle('Sentiment Distribution for Top 10 Comment Columns', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# Visualization 5: Polarity vs Subjectivity Scatter Plot
plt.figure(figsize=(12, 8))
# Sample for better visualization if too many points
if len(overall_polarities) > 5000:
    sample_indices = np.random.choice(len(overall_polarities), 5000, replace=False)
    sample_polarities = [overall_polarities[i] for i in sample_indices]
    sample_subjectivities = [overall_subjectivities[i] for i in sample_indices]
else:
    sample_polarities = overall_polarities
    sample_subjectivities = overall_subjectivities

plt.scatter(sample_polarities, sample_subjectivities, alpha=0.5, s=10)
plt.xlabel('Polarity (Negative to Positive)')
plt.ylabel('Subjectivity (Objective to Subjective)')
plt.title('Sentiment Analysis: Polarity vs Subjectivity')
plt.axhline(y=0.5, color='gray', linestyle='--', linewidth=1, alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Detailed Summary Report
print("="*80)
print("SENTIMENT ANALYSIS SUMMARY REPORT")
print("="*80)
print(f"\nTotal Comment Columns Analyzed: {len(comment_columns)}")
print(f"Total Non-Empty Comments: {len(all_comments)}")

print("\n" + "-"*80)
print("OVERALL SENTIMENT METRICS")
print("-"*80)
print(f"Average Polarity: {np.mean(overall_polarities):.4f}")
print(f"  - Range: [{np.min(overall_polarities):.4f}, {np.max(overall_polarities):.4f}]")
print(f"  - Standard Deviation: {np.std(overall_polarities):.4f}")

print(f"\nAverage Subjectivity: {np.mean(overall_subjectivities):.4f}")
print(f"  - Range: [{np.min(overall_subjectivities):.4f}, {np.max(overall_subjectivities):.4f}]")
print(f"  - Standard Deviation: {np.std(overall_subjectivities):.4f}")

print("\n" + "-"*80)
print("SENTIMENT LABEL DISTRIBUTION")
print("-"*80)
label_counts = pd.Series(overall_labels).value_counts()
for label, count in label_counts.items():
    percentage = (count / len(overall_labels)) * 100
    print(f"{label.capitalize()}: {count:,} ({percentage:.2f}%)")

print("\n" + "-"*80)
print("TOP 10 COMMENT COLUMNS BY COMMENT COUNT")
print("-"*80)
top_10_summary = sentiment_summary.head(10)
for col in top_10_summary.index:
    print(f"\n{col}:")
    print(f"  - Non-empty comments: {top_10_summary.loc[col, 'non_empty_comments']}")
    print(f"  - Average polarity: {top_10_summary.loc[col, 'avg_polarity']:.4f}")
    print(f"  - Positive: {top_10_summary.loc[col, 'positive_count']}, "
          f"Negative: {top_10_summary.loc[col, 'negative_count']}, "
          f"Neutral: {top_10_summary.loc[col, 'neutral_count']}")

print("\n" + "="*80)


In [ ]:
# Export sentiment summary to CSV
# Ensure results directory exists
results_dir = '../results'
os.makedirs(results_dir, exist_ok=True)

csv_path = os.path.join(results_dir, 'sentiment_analysis_summary.csv')
sentiment_summary.to_csv(csv_path)
print(f"Sentiment analysis summary exported to '{csv_path}'")
